# Getting data from your Feather kit into a Python program on the PC

Written by Eric B. Wertz (eric@edushields.com)

Last modified 29-Apr-2020

It may or not be obvious, but you can send data from your Feather hardware kit that a Python program running on your PC can read.  If you think about it, this almost _has_ to be possible, because the strings inside of ```print()``` statements in CircuitPython (CP) running on your Feather gets printed out the Serial window of Mu running on the PC.  So, clearly there's a way to get that data back to the PC.

This is not the only way to send data from the Feather to the PC, but it's probably the easiest to relate to and understand.  Some alternatives are to use some sort of (usually) non-human-readable binary protocol, or a different protocol (over USB or not) than a virtual serial communications port.

## The downside

The connection used for ```print()```-ing is exactly the one that you can use to communicate your data from the Feather to the PC.  The problem is that only one program can read this data from the Feather, and Mu grabs it if you are using the Serial window.  So, as long as you don't use Mu's Serial feature, some other program can grab this output just fine.

Usually what happens is when one program is trying to use that data stream, the other one will fail if it tries to grab it also, and give you some type of error that it's busy.  Fortunely it's very clear when this happens, rather than something weirder happening -- like just silently not receiving data.  Once one program closes its connection, another program can then open it.  So even if the Serial window _is_ open, just close it and re-start your data capturing program on the PC-side, and you'll be fine.

There is another solution which requires an additional ($5-20), special USB cable that connects directly from the Feather back to the PC.  The newer (v1.1) EduShields TriplerBaseboard has a place to (more easily) connect this cable, but I haven't even tested that part of the baseboard yet to it's unclear if the connector wiring works.  I ordered a few of these from two different vendors in China at the beginning of the year, but none of them ever got here and I'm still trying to get refunds for some of them.

So, just to be clear -- this is *not* to say that you cannot use Mu at all -- you just can't use the Serial window feature.  You can continue to use it to edit your program, save new versions, etc.  Again, just don't use Serial if you're going to have your own program capture this data instead.

The other significant downside of using they way of exchanging data is that it is a bit rate-limited.  It's probably limited to just a couple of megabits per second.  Having to convert binary data into strings and back out again probably doesn't help performance either.

## Sending the data out from the CircuitPython (CP) side

If you wanted to continuously send light sensor data to the PC, maybe to plot in JupyterLab using ```matplotlib```, you'd just run a CircuitPython program like the following on your Feather board.  Note that this will _NOT_ run in Python3 on the PC.

If you're using straight CircuitPython and not the EduShields layer for ME30, you'd be using ```board.<PIN>```, ```AnalogIn``` and its ```value``` accessor directly.

Go ahead and try this now.  Get out your ME30 kit, put this code in as your ```code.py``` and run it.

## Installing software to allow USB data capture from the PC

To capture this data on the PC side, you need a program that opens up a communications connection over USB to the kit to capture this printed-out data.  Assuming that you're going to be writing your own, you first need to install a library called "pyserial" into your Python environment.

Assuming you're using Anaconda/JupyterLab, you first have to run *Anaconda Prompt (anaconda3)* from the Start Menu on Windows, or a regular Terminal window on MacOS.

Inside of this window run ```conda install -c anaconda pyserial```. Assuming that you have an internet connection you should get some messages that lead you to believe that it was installed successfully.  Sometimes it tells you that ```conda``` is out-of-date, and you can run the suggested command after this is done.

Rather than Anaconda/JupyterLab, if you're going to use Mu to run this on the PC in Python3 Mode, I don't think that you have to install anything, because I think that Mu may have already done this for you.

However you're running Python3, you just need to be able to just run the ```import serial``` without an error, and you're ready.

## General strategy

So far you've learned two ways to get data into your program from the outside world -- ```input()``` from the keyboard, and reading data from a file (```open(), read()/readline(), close()```).  Both are kind-of similar -- each eventually yields a string, or something (bytes) that can be turned into one.

Getting data from your Feather board over the "serial port" is a third way.  Not terribly different from the two that you already know, but not substantially more difficult.

Similar to opening a file, you have to be able to state what (file) it is that you want to connect to (that is, to "open").  Windows and MacOS have different ways of naming this connection resource.  On MacOS, it's just a specially named file.  On Windows it's called a "COM port".  There's no reason why you wouldn't be able to connect multiple Feather boards to your PC, but you need to understand how to refer to each one.

My recommendation is to write the program on the PC side and initially use ```input()``` to get the data, then do whatever needs to be done.  Once that's working you just change the PC side (ideally you just change one line of code, if you do it intelligently) to call the other input function, and then run the CP program to replace the human.  Then you're pretty much done.

Note that while you lose the ability to use ```print()``` statements in the CP program for debugging, there's no such limitation on the PCs-side, because there's no program capturing its output.  So it still easy to debug/trace what's going on on the PC-side as normal.

## Determining the input file on MacOS

There's a fancier way of finding out this thing's name by using some additional functions in the _pyserial_ library, but we're going to do it the more straight-forward way, which also shows you how your PC's OS deals with external hardware more generally.

On MacOS, for every USB device that's connected, a file is automagically created in your ```/dev``` directory that gives programs access to that device.  It's usually called ```/dev/tty.usbmodem<BLAH>```.  You can either use your Finder to find it, or you can open a Terminal window and enter ```ls -l /dev/tty.usbmodem*```, and it'll tell you.  Hopefully there will only be one of these on your system, so it won't be ambiguous.  This is the (device) file that you need to open to get to your kit.

If there _is_ more than one file listed, check the write date on the files that are listed, and the one we care about should have a timestamp on it that corresponds to when your kit was last plugged-in.  That's the one that you care about.

![MacOS listing COM ports.](com-ports-macos.png "Figure 1: MacOS terminal windows showing USB serial ports before/after connecting Feather.")

## Determining the input "file" on Windows

On Windows, you have to determine the name of communications (COM) port that connects to your kit.  You have to open up the Device Manager app and there's a category called _Ports (COM and LPT)_ which *only* exists if you have at least one of these type of devices connected.

If none are connected, this sub-category doesn't even show up, as shown below.
![Windows Device Manager w/o COM ports.](com-ports-none.png "Figure 2: Windows Device Manager showing no active COM ports.")

If one or more devices are connected, you should see this instead:

![Windows Device Manager with Ports category.](com-ports-CROPPED.png "Figure 3: Windows Device Manager showing COM category.")

Click on the plus symbol next to the label to show you the list of them.  You should see something like the following.

![Windows Device Manager listing Ports.](com-portlist.png "Figure 4: Windows Device Manager showing COM ports.")

In my case, _currently_ my Feather kit is connected to ```COM8```.  I say _currently_ because after the kit is disconnected, or after a suspend/sleep or reboot, the COM port may switch to a different one, so you will have to update your code if/when it changes.

## Let's shuddup and do it

If you're not already running the light sensor CP program, do that now -- with no Serial window open in Mu.  If you're already running the CP program, ensure that the Serial window is close.  With the Serial window closed, I believe that the program will continue to generate that stream of floating point data, it's just that nothing is trying to read it.

Then run the following code on the PC-side.  You may/should just be able to run this directly in JupyterLab if that's how you're reading this right now.  _Note that you will almost certainly have to edit where setting the ```COM_DEVICEFILE``` is done.

Note that the reading process assumes that the input coming in from the serial port is line-oriented.  Data is buffered-up until an end-of-line character is reached, at which point the called function (```com_port_readline()```) returns a string.  Until a whole line is collected, this function will return ```None``` so that the caller can continue to perform other work rather than block waiting for input.

In [ ]:
# simpler interface to serial port input and demo
#
# Written by Eric B. Wertz (eric@edushields.com)
# Last updated 29-Apr-2020

import serial

COM_DEVICEFILE = "COM8"                    # Windoze example
#COM_DEVICEFILE = "/dev/tty.usbmodem146201" # macOS example

ser = serial.Serial(COM_DEVICEFILE, timeout=0.0)

g_ignore_newline = False
g_buffer = ""

def com_port_readline():
    global g_ignore_newline, g_buffer

    while True:
        b = ser.read(1)
        if len(b) == 0:
            return None
        c = b.decode("utf-8")

        if c == '\r':
            s = g_buffer
            g_buffer = ""
            g_ignore_newline = True           # EOL = "\r\n"
            return s

        if c == '\n':
            if g_ignore_newline:
                g_ignore_newline = False
            else:
                s = g_buffer
                g_buffer = ""
                g_ignore_newline = False
                return s
        else:
            g_ignore_newline = False
            g_buffer += c

    return  None  # NOTREACHED

# FROM HERE DOWN IS THE TEST/DEMO
while True:
    s = com_port_readline()
    if s is not None:
        print(s)


hello21
hello22
hello23
hello24
hello25
hello26
hello27
hello28
hello29
hello30
hello31
hello32
hello33
hello34
hello35
hello36
hello37
hello38
hello39
hello40
hello41
hello42
hello43
hello44
hello45
hello46
hello47
hello48
hello49
hello50
hello51
hello52
hello53
hello54
hello55
hello56
hello57
hello58
hello59
hello60
hello61
hello62
hello63
hello64
hello65
hello66
hello67
hello68
hello69
hello70
hello71
hello72
hello73
hello74
hello75
hello76
hello77
hello78
hello79
hello80
hello81
hello82
hello83
hello84
hello85
hello86
hello87
hello88
hello89
hello90
hello91
hello92
hello93
hello94
hello95
hello96
hello97
hello98
hello99
hello100
hello101
hello102
hello103
hello104
hello105
hello106
hello107
hello108
hello109
hello110
hello111
hello112
hello113
hello114
hello115
hello116
hello117
hello118
hello119
hello120
hello121
hello122
hello123
hello124
hello125
hello126
hello127
hello128
hello129
hello130
hello131
hello132
hello133
hello134
hello135
hello136
hello137
hello138
hello139
hello140

## Getting scary-fancy

You can theoretically use any input device on the kit to control a program on the PC from "any" microcontroller this way, as long as the host program knows how to read strings in from a COM port.  This often isn't the case for a lot of programs out there that only know how to get their input from either a "real" data file or from a USB keyboard. 

It's also possible to have your Feather directly emulate a USB mouse or USB keyboard using the ```Mouse``` or ```Keyboard``` class in CircuitPython, but that's different from this method and I won't discuss it here.  However, if you want to learn more about that, see https://learn.adafruit.com/circuitpython-essentials/circuitpython-hid-keyboard-and-mouse.  YOu can use any combination of input devices from the kit to control the mouse to fake moving in the X and Y axes.

Using this Keyboard class really does emulate a human being typing on the keyboard, so your program (or any other) can then use ```input()``` to read data.  The PC can't tell the difference between your home-made keyboard and/or mouse and one that you bought.  Noice!